In [119]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)
#reload (den)
reload(info)

<module 'src.info' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\info.py'>

In [120]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime
import src.info as info
import src.in_out as io
import numpy as np
from src.density import Density, draw

1. **data** klasöründeki tüm dosya yolları alınıyor

In [98]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

2. *Her bir path'de bulunan veriler **dataframe** olarak alınıyor. Her bir df, **split_df** metodu ile saatlik olarak **23** ayrı df'e bölünüyor. Bölünmüş her bir df'in **mid_price**'ı hesaplanıp yeni bir sütun olarak **df**'e ekleniyor.* **get_info** metodu her bir path için bu verileri kullanarak bir **info** örneği oluşturup döndürür. Her bir **info** örneği, aşağıdaki özelliklere sahiptir:<br>
    * Okunan dosyanın tarihi
    * Ürünün adı
    * Okunan dosyaya ait yol
    * Tüm verileri içeren dataframe
    * Saatlik olarak bölünmüş dataframe listesi


In [99]:
infolist = [info.get_info(path) for path in path_list]

3. Burada, oluşturulan infolar arasında bir **eşleştirme** yapılmaktadır. Çünkü her bir **spread** hesabının hangi veriler kullanılarak yapılacağı bilinmelidir. Eşleşen infolar, tuple çiftleri olarak tutulmakta.
  Elimizde 2018:09:06 tarihinin 6A,6B,6C ürünlerine ait verileri olsun. Bu durumda 6A-6B, 6A-6C, 6B-6C arasında 3 adet spread hesaplanacaktır. Hesaplama öncesi bu eşleştirme, **daily_data** modülünün **to_match_days** metoduyla yapılmaktadır.

In [103]:
paired_days = dt.to_match_days(infolist)

4.  **daily_datas**: GünlükVeri örneklerinin tutulduğu liste <br />
**dt.to_match_hour** metodu ile her bir eşleşmiş çiftin mid_price'ları saatlik olarak alınıp birleştiriliyor. <br />
**gunluk_veri**: **tarih**, her bir saate ait **change**,**spread**,**amplitude**,**duration**,**urun adları**...vs içeriyor <br />
**paired_hours:** 23 adet df içerir. her bir df mid_price çifti içerir

In [121]:
daily_data = []

In [122]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product   # urun adi
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.DailyData(mid_price_list=paired_hours,date=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    daily_data.append(gunluk_veri)

5. Her bir **DailyData** örneği ürünlere göre ayrı ayrı gruplandırılıyor. <br>
    (['**6AU8_6BU8**': [gunluk veriler...], '**6AU8_6CU8**':[gunluk veriler...], '**6BU8_6CU8**':[gunluk veriler...]])

In [133]:
products={}

In [134]:
for gunluk in daily_data:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:
        products[gunluk.name] = [gunluk]

In [135]:
_products = products.copy()

**combined_days :**  key-value olarak ürün adları ve verileri içerir <br />
**combined_days_detail :** key-value olarak ürün adları ve verilere ait istatistiksel bilgileri içerir <br />
**combined_days_con :** key-value olarak ürün adları ve median koşuluna göre yeniden düzenlenmiş verileri içerir <br />
* 6BU8_6CU8'e ait verilere erişmek için:<br />
 ```python
combined_days_detail['6BU8_6CU8']
```
* 6BU8_6CU8'e ait 18:00:00-19:00:00 saatleri arası verilere erişmek için: <br />
 ```python
combined_days['6BU8_6CU8'].loc[time(18)]
```

In [143]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [144]:
stop=False
i=0
while products:
    urun = products.popitem()
    df = dt.get_all_data(urun[1]).dropna()
    con_df = con.scan(df)
    combined_days[urun[0]] = df
    combined_days_con[urun[0]] = con_df
#    combined_days_detail[urun[0]] = aux.get_detail(df)
#     combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

---

# koda dahil olmayan alan

In [142]:
products['6AU8_6BU8'][0]

DailyData(mid_price_list=[          time  09/06/2018_6AU8  09/06/2018_6BU8
0     18:00:00          0.71930          1.79155
1     18:00:01          0.71930          1.29155
2     18:00:02          0.71925          1.29155
3     18:00:03          0.71925          1.29160
4     18:00:04          0.71925          1.29165
5     18:00:05          0.71935          1.29175
6     18:00:06          0.71935          1.29170
7     18:00:07          0.71935          1.29170
8     18:00:08          0.71935          1.29165
9     18:00:09          0.71935          1.29170
10    18:00:10          0.71935          1.29165
11    18:00:11          0.71935          1.29165
12    18:00:12          0.71935          1.29165
13    18:00:13          0.71935          1.29165
14    18:00:14          0.71935          1.29175
15    18:00:15          0.71935          1.29175
16    18:00:16          0.71935          1.29170
17    18:00:17          0.71935          1.29165
18    18:00:18          0.71935          1.

In [147]:
_products.popitem()

('6BU8_6CU8',
 [DailyData(mid_price_list=[          time  09/06/2018_6BU8  09/06/2018_6CU8
  0     18:00:00          1.79155          0.71930
  1     18:00:01          1.29155          0.71930
  2     18:00:02          1.29155          0.71925
  3     18:00:03          1.29160          0.71925
  4     18:00:04          1.29165          0.71925
  5     18:00:05          1.29175          0.71935
  6     18:00:06          1.29170          0.71935
  7     18:00:07          1.29170          0.71935
  8     18:00:08          1.29165          0.71935
  9     18:00:09          1.29170          0.71935
  10    18:00:10          1.29165          0.71935
  11    18:00:11          1.29165          0.71935
  12    18:00:12          1.29165          0.71935
  13    18:00:13          1.29165          0.71935
  14    18:00:14          1.29175          0.71935
  15    18:00:15          1.29175          0.71935
  16    18:00:16          1.29170          0.71935
  17    18:00:17          1.29165         

In [145]:
combined_days.keys()

dict_keys(['6BU8_6CU8', '6AU8_6CU8', '6AU8_6BU8'])

In [154]:
combined_days['6AU8_6BU8']

duration  amplitude
18:00:00 0         1.0  31250.000
         1         3.0    -11.250
         2         4.0     10.000
         3         1.0     -3.125
         4         4.0      3.125
         5         2.0     -6.250
         6         2.0      6.250
         7        52.0     -6.250
         8         1.0      5.000
         9         2.0     -8.125
         10        3.0      6.250
         11        5.0     -3.125
         12        4.0      3.125
         13       25.0     -3.125
         14        2.0      5.000
         15        1.0     -5.000
         16        3.0      5.000
         17        6.0     -5.000
         18        4.0      5.000
         19      138.0    -10.000
         20      210.0     20.625
         21       14.0     -3.125
         22       25.0      6.250
         23       46.0     -6.250
         24      136.0      8.125
         25       56.0     -5.000
         26        1.0      3.125
         27       63.0     -6.250
         28      121.0      6.250
         29       69.0     -3.125
...                ...        ...
16:00:00 372      14.0     -3.125
         373      15.0      3.125
         374      14.0    -10.000
         375      51.0     10.000
         376       2.0     -3.125
         377      15.0      3.125
         378      16.0    -26.250
         379       4.0      6.250
         380       2.0    -12.500
         381       3.0     16.250
         382       1.0     -6.250
         383       8.0      9.375
         384       4.0     -3.125
         385      32.0      6.250
         386       4.0    -36.250
         387       3.0     15.625
         388       3.0     -9.375
         389       1.0      6.250
         390      12.0    -15.625
         391       1.0      6.250
         392       1.0     -3.125
         393       5.0      9.375
         394      11.0    -11.250
         395       1.0      3.125
         396       2.0     -3.125
         397       6.0      9.375
         398       3.0     -3.125
         399       1.0      5.000
         400       4.0     -6.250
         401       1.0      3.125

[21223 rows x 2 columns]

In [152]:
combined_days['6AU8_6BU8'].index.levels[0].tolist()

[datetime.time(18, 0),
 datetime.time(19, 0),
 datetime.time(20, 0),
 datetime.time(21, 0),
 datetime.time(22, 0),
 datetime.time(23, 0),
 datetime.time(0, 0),
 datetime.time(1, 0),
 datetime.time(2, 0),
 datetime.time(3, 0),
 datetime.time(4, 0),
 datetime.time(5, 0),
 datetime.time(6, 0),
 datetime.time(7, 0),
 datetime.time(8, 0),
 datetime.time(9, 0),
 datetime.time(10, 0),
 datetime.time(11, 0),
 datetime.time(12, 0),
 datetime.time(13, 0),
 datetime.time(14, 0),
 datetime.time(15, 0),
 datetime.time(16, 0)]

In [156]:
combined_days_con['6AU8_6BU8'].loc[time(6, 0)]

,duration,amplitude
0,1.0,-6.875
1,3.0,10.000
4,18.0,-13.750
5,6.0,12.500
8,20.0,-6.250
11,11.0,6.250
14,20.0,-6.250
15,2.0,6.250
18,9.0,-6.250
19,12.0,6.250


In [157]:
current_medyan = dt.divide(combined_days_con['6AU8_6BU8'].loc[time(6, 0)])

In [158]:
current_medyan

{'pozitive': duration      9.5
 amplitude    10.0
 dtype: float64, 'negative': duration     9.0000
 amplitude   -9.6875
 dtype: float64}

In [164]:
con._conditionally_scan(combined_days_con['6AU8_6BU8'].loc[time(6, 0)],current_medyan)

,duration,amplitude
0,1.0,-6.875
1,3.0,10.000
4,18.0,-13.750
5,6.0,12.500
8,20.0,-6.250
11,11.0,6.250
18,31.0,-6.250
19,12.0,6.250
22,38.0,-12.500
25,30.0,20.000


---

Hesaplanmış ürün adları ve saat bilgisi alınıyor

In [159]:
all_products = list(combined_days.keys())

In [160]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

all_density:
```json
{"6BU8_6CU8": {"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 "6AU8_6CU8":{"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 ....}
```

In [161]:
all_density= {}

In [162]:
for product in all_products:
    densities = {}
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        densities[hour]= Density(df)
    all_density[product] = densities

--------------------------------


# Dosya Yazma

In [20]:
a = all_density['6BU8_6CU8'][time(18)]

In [31]:
df = a.marginal_distribution

In [33]:
io.write_excel('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\out\\',df,'18_00_00','6bu8')

In [3]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
   ...:                     'B': ['B0', 'B1', 'B2', 'B3'],
   ...:                     'C': ['C0', 'C1', 'C2', 'C3'],
   ...:                     'D': ['D0', 'D1', 'D2', 'D3']},
   ...:                    index=[0, 1, 2, 3])

# Grafik Çizme

In [42]:
dens = all_density['6AU8_6BU8'][time(20)]

In [43]:
kosullu = dens.get_distribution(choice='amplitude')

In [44]:
df = kosullu.reset_index()

In [46]:
df.amplitude.max()

21.25000000000876

In [47]:
den.draw(df)